# Imports

In [ ]:
!pip install transformers

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd


In [ ]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm
import transformers

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

In [ ]:
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Functions

In [ ]:
def clean_br(text): 
    text = re.sub(r'<h2 class="mg-card__title">+', " ", text)
    text = re.sub(r'</h2>+','.',text)
    text = re.sub('\n', '', text)
    return text

In [ ]:
def del_xa(text):

    text = re.sub(r'\xa0', " ", text)

    return text

In [ ]:
def clean(lst):  
  for i in range(len(lst)):
    lst[i] = str(lst[i])[(str(lst[i]).index('>')+1):(str(lst[i]).rindex('<'))]


# Data

Unpacking the data

In [ ]:
!unzip /content/drive/MyDrive/Colab_Notebooks/news/news_1.zip # change the path

In [ ]:
econ = pd.read_csv('/content/news_1/econ')
econ = list(econ.iloc[:,1])
clean(econ)
for i in range(len(econ)):
  econ[i] = clean_br(econ[i]).strip()

In [ ]:
polit = pd.read_csv('/content/news_1/polit')
polit = list(polit.iloc[:,1])
clean(polit)
for i in range(len(polit)):
  polit[i] = clean_br(polit[i]).strip()

In [ ]:
sc = pd.read_csv('/content/news_1/sc')
sc = list(sc.iloc[:,1])
clean(sc)
for i in range(len(sc)):
  sc[i] = clean_br(sc[i]).strip()

In [ ]:
sp = pd.read_csv('/content/news_1/sp')
sp = list(sp.iloc[:,1])
clean(sp)
for i in range(len(sp)):
  sp[i] = clean_br(sp[i]).strip()

In [ ]:
all_news = ' '.join(polit) + ' '.join(econ) + ' '.join(sc) + ' '.join(sp)

In [ ]:
len(all_news)

24393

# Tokenizer

In [ ]:
tokens = tokenizer.encode(all_news, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

5339


array([ 1295,  9853,   401, 25824, 15734,  2206,  3193,  7295, 35436,
        2665])

In [ ]:
l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

5339 4260 1065


# Transformer

In [ ]:
# torch.load(model, '/content/drive/MyDrive/Colab_Notebooks/news/news_gpt3.pt') #uncomment if you already have learned tho model

In [ ]:
from transformers import GPT2LMHeadModel, AdamW


model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);

In [ ]:
batch_size = 8
max_len = 70
epochs = 5

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

7 1


In [ ]:
def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                             token_type_ids=None, 
                            #  attention_mask=batch_mask,
                             labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                # attention_mask=batch_mask,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/5 : training


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 1/5 : validation


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 2/5 : training


...

  0%|          | 0/1 [00:00<?, ?it/s]

epoch 5/5 : training


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 5/5 : validation


  0%|          | 0/1 [00:00<?, ?it/s]

# Generate

In [ ]:
def generate_news(start_words):
  start_words = tokenizer.encode(start_words, return_tensors='pt').to(device)
  out = model.generate(
    input_ids=start_words,
    max_length=50,
    num_beams=5,
    do_sample=True,
    temperature=7.,
    top_k=55,
    top_p=0.6,
    no_repeat_ngram_size=4,
    num_return_sequences=1,
    ).cpu().numpy()
  return del_xa(re.sub('\n', '', tokenizer.decode(out[0])))


Try to generate news

In [ ]:
generate_news('В россии ')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2158: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


'В россии   В начале мая 2015 Путин в Сочи впервые предложил России помощь в расследовании катастрофы с крушением Audi «В России обсудили вопрос развития оборонки«Россия, которая должна защищать людей» «Путин предложил НАТО начать войну с'

In [ ]:
# torch.save(model, '/content/drive/MyDrive/Colab_Notebooks/news/news_gpt3.pt') # uncomment to save model after training

In [ ]:

# quite good params for model
# max_length=50,
#     num_beams=5,
#     do_sample=True,
#     temperature=8.,
#     top_k=55,
#     top_p=0.6,
#     no_repeat_ngram_size=4,
#     num_return_sequences=1,
#     ).cpu().numpy()

 # Interface

In [ ]:
pip install gradio

In [ ]:
import gradio as gr

In [ ]:
iface = gr.Interface(fn=generate_news, inputs=["text"], outputs="text").launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://16046.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app
